In [1]:
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

# dataset = OneDDatasetBuilder(
#     raw_dir='/data1/tam/datasets',
#     root_dir='/data1/tam/downloaded_datasets_new_test',
#     data_names=['CEMENT011'],
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type=torch.float32
# )

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
# dataset[0]

In [ ]:
normed_dataset = dataset.normalizing()
# normed_dataset.scaler('flowrate').lambdas_

In [2]:
batched_dataset = dataset.batching(
    batch_size = 100,
    batch_n_times = 50, 
    recursive = True, 
    sub_dir='/normed_and_batched_100_50', 
    step=1
)

In [ ]:
batched_dataset[1000]

In [ ]:
import matplotlib.pyplot as plt

normed_dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)

data = normed_dataset[0]
print(data)

fig, ax = plt.subplots(3,2)
fig.set_figheight(15)
fig.set_figwidth(10)

ax[0,0].hist(data.node_attr[:, 0], bins=100)
ax[0,0].set_title('coordinate_x')
ax[0,1].hist(data.node_attr[:, 1], bins=100)
ax[0,1].set_title('coordinate_y')

ax[1,0].hist(data.edge_attr[:, 0], bins=100)
ax[1,0].set_title('length')
ax[1,1].hist(data.edge_attr[:, 5], bins=100)
ax[1,1].set_title('volume_1')

ax[2,0].hist(data.pressure.flatten(), bins=100)
ax[2,0].set_title('pressure')
ax[2,1].hist(data.flowrate.flatten(), bins=100)
ax[2,1].set_title('flowrate')

In [ ]:
y = dataset[0].edge_attr

i=1
print(y[:,i])

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import PowerTransformer, MinMaxScaler, RobustScaler, QuantileTransformer, StandardScaler

transform1 = PowerTransformer()
transform1.fit(y)
transform1.lambdas_=[-0.24738493, -3.92501757,  1.07366881,  0.31679522,  1.95672088, -0.19925361, -0.16171953,  0.75660708]
# transform1.lambdas_[1] = -1.
y1 = transform1.transform(y)

plt.xlim([-20,20])
plt.hist(y[:,i], bins=100)
plt.show()

# plt.xlim([-20,20])
plt.hist(y1[:,i], bins=10)
plt.show()
plt.plot(y1[:,i])
plt.show()

# plt.xlim([-20,20])
# plt.hist(y2, bins=50)
# plt.show()

In [ ]:
var = []
for i in range(dataset.len()):
    data = dataset[i].edge_attr
    var.append(data)
var = torch.cat(var, dim=0)

from sklearn.preprocessing import PowerTransformer
transform = PowerTransformer()
transform.fit(var)
print(transform.lambdas_)
_var = transform.transform(var)


# from sklearn.preprocessing import MinMaxScaler
# transform = MinMaxScaler()
# transform.fit(var)
# _var = transform.transform(var)
# _var.min(), _var.max()

var0 = dataset[0].edge_attr
_var0 = transform.transform(var0)

In [ ]:
print(_var.mean(axis=0), _var.var(axis=0), _var.min(axis=0), _var.max(axis=0))
import matplotlib.pyplot as plt
y = _var[:,1]
plt.hist(y, bins=100)
plt.show()

print(_var0.mean(axis=0), _var0.var(axis=0), _var0.min(axis=0), _var0.max(axis=0))
y0 = _var0[:,1]
plt.hist(y0, bins=100)
plt.show()

In [ ]:
import pickle
from sklearn.preprocessing import MinMaxScaler

x = dataset[0].node_attr
scaler = MinMaxScaler()
scaler.fit(x)

with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable

y = dataset[0].flowrate


    
f = torch.gradient(y, spacing=4.8/200, dim=1)[0]
f = torch.clamp(f, min=-1, max=1)

import matplotlib.pyplot as plt
# plt.ylim([-10,10])
plt.plot(y[100])
plt.show()

# plt.ylim([-10,10])
plt.plot(f[100])
plt.show()
f.size(), y.size()

In [ ]:
# load data
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from dataset import OneDDatasetBuilder, OneDDatasetLoader

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
data = dataset[0]

# load and test model
from networks import MeshGraphNet

model = MeshGraphNet(
    node_in = (data.node_attr.size(1), True),
    node_out = (2, True), 
    edge_in = (data.edge_attr.size(1), True),
    edge_out = (0, False),
    hidden_size = 128,
    n_hiddens = 10,
    aggregation = 'sum'
)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5, weight_decay=1e-3)

x = dataset[0].node_attr.float()
edge_index = dataset[0].edge_index
edge_attr = dataset[0].edge_attr.float()

print(x.size(), edge_attr.size())

# out_node, out_edge = model.forward(x=x, edge_index=edge_index, edge_attr=edge_attr)

# print(out_node.size())

# calculate derivative
T = 4.8
nt = data.number_of_timesteps
pressure_grad = torch.gradient(data.pressure, spacing=T / (nt-1), dim=1)[0]
flowrate_grad = torch.gradient(data.flowrate, spacing=T / (nt-1), dim=1)[0]
print(pressure_grad.size(), flowrate_grad.size())
F_grad_timestep = torch.cat([pressure_grad[:,50].unsqueeze(1), flowrate_grad[:,50].unsqueeze(1)], dim=1)
# F_grad_timestep = torch.clamp(F_grad_timestep, min=-1, max=1)
F_grad_timestep = F_grad_timestep.float()
print(F_grad_timestep.size())
F_timestep_prev = torch.cat([data.pressure[:,49].unsqueeze(1), data.flowrate[:,49].unsqueeze(1)], dim=1)
F_timestep = torch.cat([data.pressure[:,50].unsqueeze(1), data.flowrate[:,50].unsqueeze(1)], dim=1)
print(F_timestep.size())

# loss
mse = torch.nn.MSELoss()

for i in range(500):
    out_node, out_edge = model.forward(x=x, edge_index=edge_index, edge_attr=edge_attr)
    grad_loss = mse(F_grad_timestep, out_node)

    out_F = F_timestep_prev + out_node * (T / (nt-1))
    value_loss = mse(F_timestep, out_F)

    loss = value_loss
    loss.backward()
    optimizer.step()
    print(loss.item())

In [ ]:
import matplotlib.pyplot as plt

field = 0

y1 = F_grad_timestep[:,field]
y2 = out_node[:,field].detach()
plt.plot(y1)
plt.plot(y2)
plt.show()

plt.hist(y1, bins=50)
plt.hist(y2, bins=50)
plt.show()

y3 = F_timestep[:,field]
y4 = out_F[:,field].detach()
plt.plot(y3)
plt.plot(y4)
plt.show()

plt.hist(y3, bins=50)
plt.hist(y4, bins=50)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
ending_node_id = 40000

current_node_id = ending_node_id
path = []
while True:
    path.append(current_node_id)
    current_edge_ids = np.where(edge_index[1] == current_node_id)[0]
    if len(current_edge_ids) < 1:
        break
    current_edge_id = current_edge_ids[0]
    current_node_id = edge_index[0][current_edge_id]

path = list(reversed(path))
plt.plot([y3[i] for i in path])
plt.plot([y4[i] for i in path])
plt.show()

In [ ]:
import sys
sys.path.insert(1, '/home/mlfm/tam/python_graph_utilities_v2/Codes/')
import torch
from networks import PARC
from dataset import OneDDatasetBuilder, OneDDatasetLoader

model = PARC(
    n_fields=2,
    n_timesteps=201,
    n_meshfields=(3,8), # Tuple(n_node_fields, n_mesh_fields)
    n_bcfields=0,
    n_hiddenfields=32,
    n_hiddens=10
)

dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_new',
    sub_dir='normalized',
    data_names='all',
    time_names=[str(i).zfill(3) for i in range(201)]
)
data = dataset[0]
mesh_features = (data.node_attr.float(), data.edge_attr.float())
edge_index = data.edge_index
F_initial = torch.cat([data.pressure[:,0].unsqueeze(1), data.flowrate[:,0].unsqueeze(1)], dim=-1).float()

out, out_dot = model.forward(
    F_initial = F_initial,
    mesh_features = mesh_features,
    edge_index = edge_index,
    F_boundary = None,
    timesteps = 4.8 / 200
)
out.size()